In [1]:

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
%run MLProject.ipynb

df=pd.read_csv('usgs_main.csv')
df1=df
df1['time'] = pd.to_datetime(df1['time'])
df1 = df1.sort_values('time')#KRONOLOJİK
df1 = df1.dropna(subset=['latitude','longitude','depth','mag','time'])#EKSİK DEĞERLERİ ATIŞ
df1['lats'] = np.floor(df1['latitude']).astype(int)
df1['lons'] = np.floor(df1['longitude']).astype(int)
df1.set_index('time')
dfweek = df1.set_index('time').resample('W').apply({
    'mag':'mean',
    'latitude':'mean',
    'longitude':'mean',
    'depth':'mean',
})
dfweek = dfweek.reset_index(drop=True)
dfweek.index = dfweek.index + 1
dfweek.index.name = 'timeindex'

Verileri zamana göre sıraladık,koordinatlarını,derinliğini,büyüklüğünü ve zamanını elde tuttuk

In [2]:
dfweek['futuremag']=dfweek['mag'].shift(-1)
dfweek['futuredepth']=dfweek['depth'].shift(-1)
dfweek['futurelat']=dfweek['latitude'].shift(-1)
dfweek['futurelon']=dfweek['longitude'].shift(-1)
dfweek=dfweek.dropna(subset=['futuremag','futuredepth','futurelat','futurelon'])#eksikleri atalım
Y=dfweek[['futuremag','futuredepth','futurelat','futurelon']]
Y=Y.dropna().reset_index(drop=True)
hiperparametreler = {
    'n_estimators': [50, 100, 150,200,250],
    'max_depth': [4, 8, 10,12,13],
    'shiftnum': [2, 3, 4,5,6],
    'min_samples_split': [10, 20, 23,24,28],
    'max_features': [3, 5,6,7,8]
}
train = dfweek[:int((4*len(dfweek))/5)]
test = dfweek[int(4*len(dfweek)/5):]
splitter=TimeSeriesSplit(n_splits=3) # altta gridsearch ile ilgili yazı linki var,burada verbose ve n_jobs değişkenlerine koymam gerekenleri öğrendim
grid = GridSearchCV( #https://www.mygreatlearning.com/blog/gridsearchcv.
    estimator=SklearnEarthquakePredictor(random_state=42),
    param_grid=hiperparametreler,
    cv=splitter,
    scoring='neg_mean_squared_error',
    n_jobs=-1, #-1 ken bütün işlemciler kullanılıyormuş
    verbose=1  #aşamaları yazdırır 
)
grid.fit(train)
best_model = grid.best_estimator_
test_predictions = best_model.predict(test)
test_actual = test[['futuremag', 'futuredepth', 'futurelat', 'futurelon']].dropna()
min_len = min(len(test_predictions), len(test_actual))
test_mse = mean_squared_error(test_actual.iloc[:min_len].values, test_predictions[:min_len])
target_names = ['futuremag', 'futuredepth', 'futurelat', 'futurelon']
target_labels = ['Magnitude', 'Depth', 'Latitude', 'Longitude']
#Tüm işlemler yapılınca MAE MSE R2 gibi metriklerin döngüsünü yapay zekaya yazdırdım
for i, (name, label) in enumerate(zip(target_names, target_labels)):
    if i < test_predictions.shape[1] and i < test_actual.shape[1]:
        target_mse = mean_squared_error(
            test_actual.iloc[:min_len, i], 
            test_predictions[:min_len, i]
        )
        target_mae = mean_absolute_error(
            test_actual.iloc[:min_len, i], 
            test_predictions[:min_len, i]
        )
        target_r2 = r2_score(
            test_actual.iloc[:min_len, i], 
            test_predictions[:min_len, i]
        )
        
        print(f"   {label:12}: MSE={target_mse:.3f}, MAE={target_mae:.3f}, R²={target_r2:.3f}")

Fitting 3 folds for each of 3125 candidates, totalling 9375 fits
   Magnitude   : MSE=0.036, MAE=0.164, R²=-2.731
   Depth       : MSE=11.905, MAE=2.709, R²=-0.970
   Latitude    : MSE=1.788, MAE=1.086, R²=-0.160
   Longitude   : MSE=15.720, MAE=3.022, R²=0.144


c:\Users\eren1\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan nan nan ... nan nan nan]
  warnings.warn(


In [ ]:
import joblib
import json

joblib.dump(best_model, 'models/random_forest_dataset1.pkl')#kaydetmemde yapay zeka yardımcı oldu
rf_best_params = {
    'dataset1_params': grid.best_params_
}
print(grid.best_params_)

{'max_depth': 4, 'max_features': 3, 'min_samples_split': 10, 'n_estimators': 50, 'shiftnum': 2}


RANDOM FOREST-DATASET2
Çok benzer,sadece gerekli ufak değişiklikler yapıldı,hiperparametre değerleri değiştirildi

In [4]:
pn=pd.read_csv('Significant Earthquake Dataset 1900-2023.csv')
df2=pn
df2=df2.rename(columns={'Time':'time','Mag':'mag','Depth':'depth','Latitude':'latitude','Longitude':'longitude' })#küçük harf uyuşmazlığı olmasın diye
df2['time'] = pd.to_datetime(df2['time'])
df2 = df2.sort_values('time')
df2 = df2.dropna(subset=['latitude','longitude','depth','mag','time'])
df2['lats'] = np.floor(df2['latitude']).astype(int)
df2['lons'] = np.floor(df2['longitude']).astype(int)
dfyear = df2.set_index('time').resample('YE').apply({
    'mag':'mean',
    'latitude':'mean',
    'longitude':'mean',
    'depth':'mean',
})
dfyear = dfyear.reset_index(drop=True)
dfyear.index = dfyear.index + 1
dfyear.index.name = 'timeindex'
dfyear['futuremag']=dfyear['mag'].shift(-1)
dfyear['futuredepth']=dfyear['depth'].shift(-1)
dfyear['futurelat']=dfyear['latitude'].shift(-1)
dfyear['futurelon']=dfyear['longitude'].shift(-1)
dfyear=dfyear.dropna(subset=['futuremag','futuredepth','futurelat','futurelon'])
Y_2=dfyear[['futuremag','futuredepth','futurelat','futurelon']]
Y_2=Y_2.dropna().reset_index(drop=True)

In [5]:
hiperparametreler_2 = {
    'n_estimators': [100,200,300,400,500],
    'max_depth': [4, 9, 11,14,15],
    'shiftnum': [4, 5, 6,7,8],
    'min_samples_split': [10, 20,30,35,45],
    'max_features': [3, 5,6,8,10]
}
train_2 = dfyear[:int((4*len(dfyear))/5)]
test_2 = dfyear[int(4*len(dfyear)/5):]
splitter_2=TimeSeriesSplit(n_splits=3)
grid_2 = GridSearchCV(
    estimator=SklearnEarthquakePredictor(random_state=42),
    param_grid=hiperparametreler_2,
    cv=splitter_2,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)
grid_2.fit(train_2)
best_model_2 = grid_2.best_estimator_
test_predictions_2 = best_model_2.predict(test_2)
test_actual_2 = test_2[['futuremag', 'futuredepth', 'futurelat', 'futurelon']].dropna()
min_len = min(len(test_predictions_2), len(test_actual_2))
test_mse_2 = mean_squared_error(test_actual_2.iloc[:min_len].values, test_predictions_2[:min_len])
target_names = ['futuremag', 'futuredepth', 'futurelat', 'futurelon']
target_labels = ['Magnitude', 'Depth', 'Latitude', 'Longitude']
for i, (name, label) in enumerate(zip(target_names, target_labels)):
    if i < test_predictions_2.shape[1] and i < test_actual_2.shape[1]:
        target_mse = mean_squared_error(
            test_actual_2.iloc[:min_len, i], 
            test_predictions_2[:min_len, i]
        )
        target_mae = mean_absolute_error(
            test_actual_2.iloc[:min_len, i], 
            test_predictions_2[:min_len, i]
        )
        target_r2 = r2_score(
            test_actual_2.iloc[:min_len, i], 
            test_predictions_2[:min_len, i]
        )
        
        print(f"   {label:12}: MSE={target_mse:.3f}, MAE={target_mae:.3f}, R²={target_r2:.3f}")

Fitting 3 folds for each of 3125 candidates, totalling 9375 fits
   Magnitude   : MSE=0.001, MAE=0.017, R²=-0.821
   Depth       : MSE=205.904, MAE=12.962, R²=-0.741
   Latitude    : MSE=15.185, MAE=3.078, R²=-0.271
   Longitude   : MSE=221.888, MAE=12.331, R²=-0.415


c:\Users\eren1\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan nan nan ... nan nan nan]
  warnings.warn(


In [9]:
joblib.dump(best_model_2, 'models/random_forest_dataset2.pkl')
rf_best_params['dataset2_params'] = grid_2.best_params_
print(grid_2.best_params_)

{'max_depth': 4, 'max_features': 3, 'min_samples_split': 10, 'n_estimators': 100, 'shiftnum': 4}


RANDOM FOREST EXTRA-TİME STEP SAYISINI ARTTIRIP HİPERPARAMETRE DEĞERLERİNİ KÜÇÜLTEREK DAHA İYİ BİR SONUÇ ARAYACAĞIM

In [7]:
hiper=pd.read_csv('Significant Earthquake Dataset 1900-2023.csv')
rfext=hiper
rfext=rfext.rename(columns={'Time':'time','Mag':'mag','Depth':'depth','Latitude':'latitude','Longitude':'longitude' })
rfext['time'] = pd.to_datetime(rfext['time'])
rfext = rfext.sort_values('time')
rfext = rfext.dropna(subset=['latitude','longitude','depth','mag','time'])
rfext['lats'] = np.floor(rfext['latitude']).astype(int)
rfext['lons'] = np.floor(rfext['longitude']).astype(int)
dfplus = rfext.set_index('time').resample('W').apply({
    'mag':'mean',
    'latitude':'mean',
    'longitude':'mean',
    'depth':'mean',
})
dfplus = dfplus.reset_index(drop=True)
dfplus.index = dfplus.index + 1
dfplus.index.name = 'timeindex'
dfplus['futuremag']=dfplus['mag'].shift(-1)
dfplus['futuredepth']=dfplus['depth'].shift(-1)
dfplus['futurelat']=dfplus['latitude'].shift(-1)
dfplus['futurelon']=dfplus['longitude'].shift(-1)
dfplus=dfplus.dropna(subset=['futuremag','futuredepth','futurelat','futurelon'])
Y_2=dfplus[['futuremag','futuredepth','futurelat','futurelon']]
Y_2=Y_2.dropna().reset_index(drop=True)

In [8]:
hiperparametreler_yen = {
    'n_estimators': [50,60,70,80,90],
    'max_depth': [3, 5, 8,11,13],
    'shiftnum': [2, 3, 5,7,9],
    'min_samples_split': [15, 18,22,32,40],
    'max_features': [4,5,6,7,8]
}
train_art = dfplus[:int((4*len(dfplus))/5)]
test_art = dfplus[int(4*len(dfplus)/5):]
splitter_art=TimeSeriesSplit(n_splits=3)
grid_max = GridSearchCV(
    estimator=SklearnEarthquakePredictor(random_state=42),
    param_grid=hiperparametreler_yen,
    cv=splitter_art,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)
grid_max.fit(train_art)
best_model_plus = grid_max.best_estimator_
test_predictions_ex = best_model_plus.predict(test_art)
test_actual_ex = test_art[['futuremag', 'futuredepth', 'futurelat', 'futurelon']].dropna()
min_len = min(len(test_predictions_ex), len(test_actual_ex))
target_names = ['futuremag', 'futuredepth', 'futurelat', 'futurelon']
target_labels = ['Magnitude', 'Depth', 'Latitude', 'Longitude']
for i, (name, label) in enumerate(zip(target_names, target_labels)):
    if i < test_predictions_ex.shape[1] and i < test_actual_ex.shape[1]:
        target_mse = mean_squared_error(
            test_actual_ex.iloc[:min_len, i], 
            test_predictions_ex[:min_len, i]
        )
        target_mae = mean_absolute_error(
            test_actual_ex.iloc[:min_len, i], 
            test_predictions_ex[:min_len, i]
        )
        target_r2 = r2_score(
            test_actual_ex.iloc[:min_len, i], 
            test_predictions_ex[:min_len, i]
        )
        
        print(f"   {label:12}: MSE={target_mse:.3f}, MAE={target_mae:.3f}, R²={target_r2:.3f}")

Fitting 3 folds for each of 3125 candidates, totalling 9375 fits
   Magnitude   : MSE=0.025, MAE=0.126, R²=-0.227
   Depth       : MSE=2208.496, MAE=35.196, R²=0.112
   Latitude    : MSE=217.520, MAE=11.554, R²=-0.203
   Longitude   : MSE=2896.299, MAE=43.082, R²=0.014


c:\Users\eren1\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:1108: UserWarning: One or more of the test scores are non-finite: [nan nan nan ... nan nan nan]
  warnings.warn(


In [11]:
joblib.dump(best_model_plus, 'models/random_forest_extra.pkl')
rf_best_params['extra_params'] = grid_max.best_params_
print(grid_max.best_params_)
with open('models/random_forest_best_params.json', 'w') as f:
    json.dump(rf_best_params, f, indent=2)

{'max_depth': 3, 'max_features': 4, 'min_samples_split': 15, 'n_estimators': 50, 'shiftnum': 2}
